# Fast IP Range Assessment
- Author: Aidan Marlin
- Date: 20220610
- Version: 1.1
- Description: Jupyter Notebook to automate the execution of key infrastructure scans

In [ ]:
ip_range = "192.168.0.0/24"

### No need to modify from here
import socket
import os

os.environ["PATH"] = os.environ["PATH"] + ":/run/current-system/sw/bin"

!mkdir -p tools/masscan
!mkdir -p tools/nmap
!mkdir -p tools/puppeteer

def read_file(location):
    with open(location) as f:
        text = f.read()
        print(text)


# Target

In [ ]:
print(ip_range)


# Pentester Details

## IP Address

In [ ]:
!curl https://ifconfig.me/


## Start Time

In [ ]:
!date


## Interface Info

In [ ]:
!ip a


# Discovery

## Ping Scan

In [ ]:
!nmap -sn -T5 {ip_range} -v0 -oA tools/nmap/internal_ping_scan


In [ ]:
read_file("tools/nmap/internal_ping_scan.gnmap")

## Fast TCP Port Scan

In [ ]:
!sudo masscan --top-ports 100 --rate=1000 -oJ tools/masscan/tcp_top_100_json.txt -oG tools/masscan/tcp_top_100_grep.txt {ip_range}


In [ ]:
read_file("tools/masscan/tcp_top_100_grep.txt")


## Fast UDP Port Scan

In [ ]:
!sudo nmap -T5 -sU --top-ports 100 -v0 {ip_range} -oA tools/nmap/udp_top_100


In [ ]:
read_file("tools/nmap/udp_top_100.gnmap")


## ivre

In [ ]:
!docker-compose -f /home/user/git/pentest-tools/ivre/docker/docker-compose.yml up -d


In [ ]:
ivre_container_id = os.popen("docker ps -f 'name=ivreclient' --format '{{.ID}}'").read().rstrip()
print(ivre_container_id)


In [ ]:
!docker exec -t $ivre_container_id ivre runscans --network $ip_range


In [ ]:
!docker exec -t $ivre_container_id ivre getwebdata


In [ ]:
!docker exec -t $ivre_container_id ivre scan2db -r /scans


In [ ]:
!docker exec -t $ivre_container_id ivre db2view


In [ ]:
import shutil
from pathlib import Path
home = str(Path.home())

os.system("docker run --add-host=host.docker.internal:host-gateway -u root --shm-size 1G --rm -v \"" + os.getcwd() + "/tools/puppeteer:/screenshots\" alekzonder/puppeteer:latest full_screenshot http://host.docker.internal/ 1024x768")
if not os.path.isfile("tools/puppeteer/full_screenshot_1024_768.png"):
    shutil.copyfile(home + "/git/nixos/resources/404.png", "tools/puppeteer/full_screenshot_1024_768.png")


![Target screenshot](tools/puppeteer/full_screenshot_1024_768.png)


Results at http://localhost/

# Exploit

## Searchsploit

In [ ]:
!docker run --rm -v "$PWD:/mnt" booyaabes/kali-linux-full searchsploit -v --nmap /mnt/tools/nmap/udp_top_100.xml 2>/dev/null


# End Time

In [ ]:
!date
